# Main executeable

Results saved to results/

## Imports & API connection test & init

In [5]:
import praw                             #Reddit API
import nltk                             #natural language
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import yfinance as yf                   #yahoo finance
from auth import *                      #authentification details - create an auth.py file in code/ with authentification details
import matplotlib.pyplot as plt
import math
import datetime as dt
import pandas as pd
import numpy as np
import requests         
from bs4 import BeautifulSoup as bs     #web scraper

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT)

print(reddit.read_only)

True


## Get resources
Scrape updated list of tickers in S&P500 

In [12]:
def get_spy():
    url = 'https://www.slickcharts.com/sp500'
    request = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs(request.text, "lxml")

    stats = soup.find('table',class_='table table-hover table-borderless table-sm')

    df = pd.read_html(str(stats))[0]
    #FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.

    df['% Chg'] = df['% Chg'].str.strip('()-%')

    df['% Chg'] = pd.to_numeric(df['% Chg'])

    df['Chg'] = pd.to_numeric(df['Chg'])

    return df

df = get_spy()
sp_tickers = df["Symbol"]   #list of tickers in sp500 as strings

print(len(sp_tickers))

#Sample output to test
for i in range(0, 10):
    print(sp_tickers[i])


503
MSFT
AAPL
NVDA
AMZN
META
GOOGL
GOOG
BRK.B
LLY
AVGO


/var/folders/1c/4fc0bzrd6sb0t89qb2xmbr740000gn/T/ipykernel_9598/1317168326.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(stats))[0]


## Functions

In [15]:
def commentSentiment(ticker, urlT):
    #given ticker and post that mentions ticker, will calculate average sentiment of comments to that post


    comments = [] 
    bodyComment = []

    #get comments from sub
    try:
        post = reddit.submission(url=urlT)
        comments = post.comments            #returns iterable CommentForest object
    except:
        return 0
    
    #save each comment to array
    for comment in comments:
        try: 
            bodyComment.append(comment.body)
            print(comment.body)    
        except:
            print("fail")
            return 0
    
    sia = SIA()
    results = []
    for line in bodyComment:
        scores = sia.polarity_scores(line)
        scores['headline'] = line

        results.append(scores)
    
    df = pd.DataFrame.from_records(results)
    df.head()
    df['label'] = 0
    
    try:
        df.loc[df['compound'] > 0.1, 'label'] = 1
        df.loc[df['compound'] < -0.1, 'label'] = -1
    except:
        return 0
    
    averageScore = 0
    position = 0
    while position < len(df.label)-1:
        averageScore = averageScore + df.label[position]
        position += 1

    averageScore = averageScore/len(df.label) 
    
    return(averageScore)

print(commentSentiment("TSLA", "https://www.reddit.com/r/wallstreetbets/comments/1c5poh3/tsla_going_below_125/"))


**User Report**| | | |
:--|:--|:--|:--
**Total Submissions** | 2 | **First Seen In WSB** | 3 years ago
**Total Comments** | 31 | **Previous Best DD** | 
**Account Age** | 3 years | | 

[**Join WSB Discord**](http://discord.gg/wsbverse)
Woods hasn’t sold her shares yet so Tesla hasn’t hit bottom yet. Remember she buys high and sells low
I will keep my mouth shut this time. Whatever I comment here was wrong, I said up it went down, I said down it went up, yes is no, stop is go….this site is cursed ![img](emote|t5_2th52|4267)
I started a position at 175 so definitely going below 125
I'd say a lull because there is no new hotness causing the investors to foam at the mouth.
https://preview.redd.it/a3aep151uwuc1.jpeg?width=1080&format=pjpg&auto=webp&s=36d05628b9b1fcead7aba6ea45bd05e429cd882e

Took a year
Stupid me sold my 4/20 put at a loss last week.
$69 by 4/20
Guys were laughing at me when I said I would start a position at $100. Same guys will sell to me because they are afraid of losin